# Exercice 2: A-stabilité méthode d'Euler
Déterminer les conditions de A-stabilité de l'Euler avant, arrière centré.

- Euler avant: $k<2$
- Euler arrière: inconditionnellement A-stable
- Euler centré: inconditionnellement A-stable

# Exercice 3: Monoticité des méthodes d'Euler

Déterminer la condition de monotonicité pour l’Euler avant, arrière et centré.

- Euler avant: $k<1$
- Euler arrière: inconditionnellement monotone
- Euler centré: $k<2$

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

tmax=10
k=2.5
N=int(np.floor(tmax/k))+1
t=np.linspace(0,tmax,N)
u_EUav=np.zeros(N)
u_EUar=np.zeros(N)
u_EUcen=np.zeros(N)

#CI
u_EUav[0]=1
u_EUar[0]=1
u_EUcen[0]=1

for i in range(N-1):
    u_EUav[i+1]=(1-k)*u_EUav[i]
    u_EUar[i+1]=u_EUar[i]/(1+k)
    u_EUcen[i+1]=((1-k/2)/(1+k/2))*u_EUcen[i]


fig,ax=plt.subplots(figsize=(5,5))
ax.plot(t,np.exp(-t))
ax.plot(t,u_EUav,'o',label='Avant')
ax.plot(t,u_EUar,'^',label='Arrière')
ax.plot(t,u_EUcen,'+',label='Centré')
ax.legend()
ax.set_title('Décroissance radioactive')
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 4: Oscillateur harmonique
L'équation de l'oscillateur harmonique est:

${\large \frac{d^{2}x}{dt^{2}}=-\frac{-K}{m}x}$

avec $K$ la constante de raideur du ressort et $m$ la masse.

Cette équation peut s'écrire sous forme matriciel:

${\large \frac{d \mathbf{x}}{dt}=M\mathbf{x}}$, avec ${\large \mathbf{x}=(p,x)}$ et 
${\large M=\begin{pmatrix} 
0 & -K\\
1/m & 0\\
\end{pmatrix}
}$

**Vérifier que le schéma Euler avant crée de l’énergie ; que l’Euler arrière en détruit ; et que seul l’Euler centré est conservatif. Illustrer ces propriétés en intégrant le système.**

In [78]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6)


def HarmOsc(K,m,t,x0,mode):
    k=t[1]-t[0]
    x=np.zeros((2,np.shape(t)[0]))
    x[:,0]=x0
    M=np.array([[0,-K],[1/m,0]])	
    if mode=='Euler avant':
        for i in range(np.shape(x)[1]-1):
            M=np.array([[0,-K],[1/m,0]])
            x[:,i+1]=(np.identity(2)+k*M).dot(x[:,i])
    elif mode=='Euler arrière':
        for i in range(np.shape(x)[1]-1):
            M=np.array([[0,-K],[1/m,0]])
            x[:,i+1]=np.linalg.solve((np.identity(2)-k*M),x[:,i])
    elif mode=='Euler centré':
        for i in range(np.shape(x)[1]-1):
            M=np.array([[0,-K],[1/m,0]])
            x[:,i+1]=np.linalg.solve((np.identity(2)-k/2*M),(np.identity(2)+k/2*M).dot(x[:,i]))
    elif mode=='Leapfrog':
        x[:,1]=x[:,0]
        for i in range (1,np.shape(x)[1]-1):
            x[:,i+1]=x[:,i-1]+2*k*M.dot(x[:,i])
    elif mode=='Heun':
        for i in range(np.shape(x)[1]-1):
            xx=x[:,i]+k*M.dot(x[:,i])
            x[:,i+1]=x[:,i]+(k/2)*(M.dot(x[:,i])+M.dot(xx))
    elif mode=='RK4':
        for i in range(np.shape(x)[1]-1):
            k1=k*M.dot(x[:,i])
            k2=k*M.dot(x[:,i]+k1/2)
            k3=k*M.dot(x[:,i]+k2/2)
            k4=k*M.dot(x[:,i]+k3)
            x[:,i+1]=x[:,i]+(k1+2*k2+2*k3+k4)/6
    elif mode=='Verlet':
        for i in range(np.shape(x)[1]-1):
            pp=x[0,i]-k*K*x[1,i]
            x[1,i+1]=x[1,i]+(k/2)*(x[0,i]/m+pp/m)
            x[0,i+1]=x[0,i]-(k/2)*(x[1,i]+x[1,i+1])
    elif mode=='Stormer_Verlet':
        for i in range(np.shape(x)[1]-1):
            pp=x[0,i]-(k/2)*k*x[1,i]
            x[1,i+1]=x[1,i]+(k)*(pp/m)
            x[0,i+1]=pp-(k/2)*(x[1,i+1])
    else:
        print('Invalid integration method')
    return(x)

def Energie_calc(x,k,m):
    return((x[0,:]*x[0,:])/(2*m)+k*(x[1,:]*x[1,:])/2)

In [43]:
#paramètres du problème
K=1.
m=1.0
#paramètre de l'intégration numérique
k=0.1
tmax=50
N=int(np.floor(tmax/k))+1
t=np.linspace(0,tmax,N)
#Conditions initiales
x0=[0,1]
#Solution analytique
omega=np.sqrt(K/m)
xsol=np.zeros((2,N))
xsol[1,:]=x0[1]*np.cos(omega*t)+x0[0]/(m*omega)*np.sin(omega*t)
xsol[0,:]=(-x0[1]*omega*np.sin(omega*t)+x0[0]/(m*omega)*omega*np.cos(omega*t))*m

x_EUav=HarmOsc(K,m,t,x0,'Euler avant')
x_EUar=HarmOsc(K,m,t,x0,'Euler arrière')
x_EUcen=HarmOsc(K,m,t,x0,'Euler centré')


E_EUcen=Energie_calc(x_EUcen,K,m)
E_EUar=Energie_calc(x_EUar,K,m)
E_EUav=Energie_calc(x_EUav,K,m)
Esol=Energie_calc(xsol,K,m)

In [47]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(10,10))
ax.plot(xsol[1,:],xsol[0,:]/m,label='Theoretical',marker='o')
ax.plot(x_EUav[1,:],x_EUav[0,:]/m,label='Euler avant')
ax.plot(x_EUar[1,:],x_EUar[0,:]/m,label='Euler arrière')
ax.plot(x_EUcen[1,:],x_EUcen[0,:]/m,label='Euler centré')
ax.plot(x0[1],x0[0]/m,'ro')
ax.legend(loc='best',fontsize='small')
ax.set_xlabel('x')
ax.set_ylabel('v')
ax.set_title('Oscillateur harmonique, dt =%.3f'%(k))
ax.set_xlim((-2,2))
ax.set_ylim((-2,2))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
%matplotlib widget
fig,ax=plt.subplots(figsize=(10,10))
ax.plot(t,E_Verl,label='Verlet')
ax.plot(t,E_StVerl,label='Stormer_Verlet')
ax.plot(t,E_Heun,label='Heun')
ax.plot(t,E_RK4,label='RK4')
ax.plot(t,E_EUcen,label='Euler centré',marker='D')
ax.plot(t,E_EUav,label='Euler avant')
ax.plot(t,E_EUar,label='Euler arrière')
ax.plot(t,Esol,label='Theoretical')
plt.ylim((0.45,0.525))
plt.xlim((-1,50))
plt.legend(loc='best',fontsize='small')
plt.xlabel('t')
plt.ylabel('Energie')
plt.title('Oscillateur harmonique: dt= %.5f'%(dt))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 6: Leap-frog

On a montré que le schéma leap-frog est inconditionnellement instable pour la décroissance radioactive

In [76]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

tmax=10
k=0.1
N=int(np.floor(tmax/k))+1
t=np.linspace(0,tmax,N)
u_LP=np.zeros(N)

#CI
u_LP[0]=1
u_LP[1]=(1-k)*u_LP[0]

for i in range(1,N-1):
    u_LP[i+1]=u_LP[i-1]-2*k*u_LP[i]



fig,ax=plt.subplots(figsize=(5,5))
ax.plot(t,np.exp(-t))
ax.plot(t,u_LP,'o',label='Leap-frog')
ax.legend()
ax.set_title('Décroissance radioactive')
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 7: Equation logistique

L'équation logistique:

${\large 
\begin{equation*}
\frac{dx}{dt}=\lambda x(1-x), \quad x_{0}=\alpha
\end{equation*}}
$

a pour solution exacte:

${\large \begin{equation*}
x(t)=\frac{\alpha}{\alpha +(1- \alpha )e^{-\lambda t}}
\end{equation*}}$

**Comparer les erreurs des schémas explicites Euler avant, Heun et RK4 pour différents pas de temps.
Le schéma Euler arrière peut s'établir analytiquement pour l'équation logistique, mais cela implique de résoudre, à chaque pas de temps, une équation du second degré, ... qui a donc deux solutions. Laquelle des deux retenirs?**

In [25]:
def int_log(N,tmax,lmb,alpha,mode):
    N=int(N)
    t=np.linspace(0,tmax,N)
    k=tmax/(float(N-1))
    F=lambda z: lmb*z*(1.-z)
    x=np.zeros(N)
    x[0]=alpha
    if mode=='Euler avant':
        for i in range(N-1):
            x[i+1]=x[i]+k*F(x[i])
    elif mode=='Heun':
        for i in range(N-1):
            xx=x[i]+k*F(x[i])
            x[i+1]=x[i]+(k/2)*(F(x[i])+F(xx))
    elif mode=='RK4':
        for i in range(N-1):
            k1=k*F(x[i])
            k2=k*F(x[i]+k1/2)
            k3=k*F(x[i]+k2/2)
            k4=k*F(x[i]+k3)
            x[i+1]=x[i]+(k1+2*k2+2*k3+k4)/6
    else:
        print('Invalid integration method')
    return(x,t)

In [38]:
lmb=1
alpha=0.1
tmax=10
k=1
N=int(np.floor(tmax/k))+1
t=np.linspace(0,tmax,N)

mode='Heun'
sol=int_log(N,tmax,lmb,alpha,mode)


fig,ax=plt.subplots(figsize=(5,5))
ax.plot(np.linspace(0,tmax,1000),alpha/(alpha+(1.-alpha)*np.exp(-lmb*np.linspace(0,tmax,1000))))
ax.plot(t,sol[0],'o',label=mode)
ax.legend()
ax.set_title('Equation logistique')
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
lmb=1
alpha=0.1
Nsim=6
N=10**(np.linspace(2,7,Nsim))
tmax=1
diff_EUav=np.zeros(Nsim)
diff_heun=np.zeros(Nsim)
diff_RK4=np.zeros(Nsim)
xexact=alpha/(alpha+(1.-alpha)*np.exp(-lmb*1.))
for j in range(Nsim):
    
    x_EUav,t=int_log(N[j],tmax,lmb,alpha,'Euler avant')
    x_heun,t=int_log(N[j],tmax,lmb,alpha,'Heun')
    x_RK4,t=int_log(N[j],tmax,lmb,alpha,'RK4')
    diff_EUav[j]=np.abs(xexact-x_EUav[-1])
    diff_heun[j]=np.abs(xexact-x_heun[-1])
    diff_RK4[j]=np.abs(xexact-x_RK4[-1])

In [92]:
fig,ax=plt.subplots(figsize=(5,5))
plt.loglog(N,diff_EUav,'bo',label='Euler avant')
plt.loglog(N,diff_heun,'rD',label='Heun')
plt.loglog(N,diff_RK4,'g^',label='RK4')
plt.loglog(N,N**(-1)*(diff_EUav[0]/N[0]**(-1)),'b--',label='$1/k$')
plt.loglog(N,(N)**(-2)*(diff_heun[0]/N[0]**(-2)),'r--',label='$1/k^{2}$')
plt.loglog(N,N**(-4)*(diff_RK4[0]/N[0]**(-4)),'g--',label='$1/k^{4}$')
plt.title('$|x(t) - x_{t/k}|,\ t=1$')
plt.xlabel('$1/k$')
plt.legend(loc='best',fontsize='small')
plt.ylim((10**(-16),10**(-1)))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 8: Oscillateur harmonique symplectique

In [79]:
#paramètres du problème
K=1.
m=1.0
#paramètre de l'intégration numérique
k=1
T=500
N=int(np.floor(T/k))+1
t=np.linspace(0,T,N)
#Conditions initiales
x0=[0,1]
#Solution analytique
omega=np.sqrt(K/m)
xsol=np.zeros((2,50000))
xsol[1,:]=x0[1]*np.cos(omega*np.linspace(0,T,50000))+x0[0]/(m*omega)*np.sin(omega*np.linspace(0,T,50000))
xsol[0,:]=(-x0[1]*omega*np.sin(omega*np.linspace(0,T,50000))+x0[0]/(m*omega)*omega*np.cos(omega*np.linspace(0,T,50000)))*m



x_Verl=HarmOsc(K,m,t,x0,'Verlet')
x_StVer=HarmOsc(K,m,t,x0,'Stormer_Verlet')
x_Heun=HarmOsc(K,m,t,x0,'Heun')
x_RK4=HarmOsc(K,m,t,x0,'RK4')
E_Verl=Energie_calc(x_Verl,K,m)
E_StVerl=Energie_calc(x_StVer,K,m)
E_RK4=Energie_calc(x_RK4,K,m)
E_Heun=Energie_calc(x_Heun,K,m)
Esol=Energie_calc(xsol,K,m)


In [80]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
ax[0].plot(xsol[1,:],xsol[0,:]/m,label='Theoretical')
ax[0].plot(x_Heun[1,:],x_Heun[0,:]/m,label='Heun')
ax[0].plot(x_Verl[1,:],x_Verl[0,:]/m,label='Verlet')
ax[0].plot(x_RK4[1,:],x_RK4[0,:]/m,label='RK4')
ax[0].plot(x0[1],x0[0]/m,'ko')
ax[0].legend(loc='best',fontsize='small')
ax[0].set_xlabel('x')
ax[0].set_ylabel('v')
ax[0].set_title('Oscillateur harmonique, k =%.3f'%(k))
ax[0].set_xlim((-2,2))
ax[0].set_ylim((-2,2))
ax[1].plot(np.linspace(0,T,50000),Esol,label='Theoretical')
ax[1].plot(t,E_Heun,label='Heun')
ax[1].plot(t,E_Verl,label='Verlet')
ax[1].plot(t,E_RK4,label='RK4')
ax[1].legend(loc='best',fontsize='small')
ax[1].set_xlabel('t')
ax[1].set_ylabel('Energie')
ax[1].set_title('Oscillateur harmonique, k =%.3f'%(k))
ax[1].set_ylim((0.4,0.6))
ax[1].set_xlim((1,50))
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 9: Couche limite

Le système suivant illustre la notion de couche limite :
    
${\large \begin{equation*}
\epsilon \frac{d^{2} y}{dx^{2}}-x^{2}\frac{dy}{dx}-y=0, \, 0<x<1, \, \epsilon =10^{-2},
\end{equation*}}$

avec les conditions frontières: $y(0)=y(1)=1$.

Condition d'unicité: $N>49$

Système linéaire à résoudre:

$
A=\begin{pmatrix}
a_{1} & c_{1} & 0 &\cdots & \cdots &  0\\
b_{2} & a_{2} & c_{2} & 0 & \cdots & 0  \\
0 & \ddots & \ddots & \ddots &  \ddots &\vdots  \\
\vdots & \ddots  & \ddots & \ddots & \ddots & 0  \\
0 & \cdots  & 0& b_{N-1} & a_{N-1} & c_{N-1}  \\
0 & \cdots  & 0 & 0 & b_{N} & a_{N} 
\end{pmatrix}
\begin{pmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{N-1}\\
y_{N}\\
\end{pmatrix}
=\begin{pmatrix}
-b_{1}\\
0\\
\vdots\\
0\\
-c_{N}\\
\end{pmatrix}
$

avec

$
\left\{
                \begin{array}{ll}
                  a_{i}=-2-\frac{h^2}{\epsilon}\\
                  b_{i}=1+\frac{hx_{i}^{2}}{2\epsilon}\\
                  c_{i}=1-\frac{hx_{i}^{2}}{2\epsilon}
                \end{array}
              \right.
$



Conditionnement de la matrice:

In [108]:
N=1
eps=10**(-2)
x=np.linspace(0.,1.,N+2)
y=np.zeros(N+2)
h=x[1]-x[0]
ybound=[1,1]
xbis=x[1:-1]
a=-2.-(h**2)/eps*np.ones(N)
b=1.+h*(xbis**2)/2./eps
c=1.-h*(xbis**2)/2./eps
A=np.diagflat(b[1:],-1)+np.diagflat(c[:-1],1)+np.diagflat(a)
#print(A)
w,v=np.linalg.eig(A.T@A)
cond=np.sqrt(np.max(np.abs(w))/np.min(np.abs(w)))
print(cond)

43421.22605507695


In [111]:

import numpy as np
import matplotlib.pyplot as plt


def tridiag(a,b,c,f):
    N=f.size
    x=np.zeros(N)
    cstar=np.zeros(N)
    astar=a[0]
    x[0]=f[0]/astar
    for k in range(1,N):
        cstar[k-1]=c[k-1]/astar
        astar=a[k]-b[k]*cstar[k-1]
        x[k]=(f[k]-b[k]*x[k-1])/astar
    for k in range(N-2,-1,-1):
        x[k]-=cstar[k]*x[k+1]
    return(x)


In [115]:
def c_lim(N):
    eps=10**(-2)
    x=np.linspace(0.,1.,N+2)
    y=np.zeros(N+2)
    h=x[1]-x[0]
    ybound=[1,1]
    xbis=x[1:-1]
    a=-2.-(h**2)/eps*np.ones(N)
    b=1.+h*(xbis**2)/2./eps
    c=1.-h*(xbis**2)/2./eps
    z=np.zeros(N)
    z[0]=-b[0]*ybound[0]
    z[-1]=-c[-1]*ybound[1]
    y[1:-1]=tridiag(a,b,c,z)
    y[0]=ybound[0]
    y[-1]=ybound[1]
    return(x,y,cond)


In [116]:
x10,y10,cond10=c_lim(10)
x20,y20,cond20=c_lim(20)
x50,y50,cond50=c_lim(50)
x120,y120,cond120=c_lim(120)



In [117]:
fig,ax=plt.subplots(figsize=(5,5))
plt.plot(x10,y10,label='N=10',linestyle='--',marker='o',ms=7)
plt.plot(x20,y20,label='N=20',linestyle='--',marker='v',ms=7)
plt.plot(x50,y50,label='N=50',linestyle='--',marker='D',ms=7)
plt.plot(x120,y120,label='N=120')
plt.legend(loc='best',fontsize='small')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 11: équation de Bratu

Résoudre l'équation de Bratu ${\large y'' = −e^y}$ , ${\large 0 < x < 1}$, avec
${\large y (0) = y (1) = 0}$. Utiliser comme premier essai ${\large y_{0}(x) = x − x^{2}}$ ;
recommencez et utilisez ${\large y_{0}(x) = 16(x − x^{2})}$. 


>**Newton-Raphson**
>
>Système d'équations non linéaires
>
>
>${\large \mathbf{F}(\mathbf{y})=\mathbf{0}}$
>
>On a besoin de deux choses pour appliquer la méthode: ${\large \mathbf{y_{0}}}$ première estimation de la solution et ${\large J}$ le jacobien de ${\large >\mathbf{F}(\mathbf{y})}$
>
>Le schéma suivant doit converger vers la solution si le système est suffisamment régulier:
>
>${\large \mathbf{y}_{k+1} = \mathbf{y}_{k} - J_{k}^{-1} \mathbf{F}_{k} \, \Rightarrow \, J_{k}\mathbf{y}_{k+1}  =J_{k}\mathbf{y}_{k}-\mathbf{F}_{k} \quad ,k=0,1,2,...}$





In [127]:
def Jac(y,h):
    N=y.size
    J=np.zeros((N,N))
    J[0,0]=-2.+(h**2)*np.exp(y[0])
    J[-1,-1]=-2.+(h**2)*np.exp(y[-1])
    J[0,1]=1.
    J[-1,-2]=1.
    for k in range(1,N-1):
        J[k,k]=-2.+(h**2)*np.exp(y[k])
        J[k,k-1]=1.
        J[k,k+1]=1.
    return(J)

def Fk(z,h):
    N=z.size
    F=np.zeros(N)
    F[0]=z[1]-2.*z[0]+(h**2)*np.exp(z[0])          # CF z_firstpoint=0
    F[-1]=-2.*z[-1]+z[-2]+(h**2)*np.exp(z[-1])# CF z_lastpoint=0
    for k in range(1,N-1):
        F[k]=z[k+1]-2.*z[k]+z[k-1]+(h**2)*np.exp(z[k])
    return(F)



def bratu(N,N_it,x,Y0):
    h=x[1]-x[0]
    Y=np.zeros((N,N_it))
    Y[:,0]=Y0
    for k in range(1,N_it):
        diag_J=-2.+(h**2)*np.exp(Y[:,k-1])
        rhs=np.dot(Jac(Y[:,k-1],h),Y[:,k-1])-Fk(Y[:,k-1],h)
        Y[:,k]=tridiag(diag_J,np.ones(N),np.ones(N),rhs)
    return(np.append(np.insert(Y[:,-1],0,0.),0.))




In [135]:
#paramètres d'intégration
N=10
N_int=5
x=np.linspace(0.,1,N+2)

#Solutions analytiques
c1=0.379
Y_anal1=-2.*np.log(np.cosh(c1*(1.-2.*x))/np.cosh(c1))
c2=2.73;
Y_anal2=-2.*np.log(np.cosh(c2*(1.-2.*x))/np.cosh(c2))

#Intégration numérique
Y1=bratu(N,N_int,x,Y0=(x[1:-1]-x[1:-1]**2))
Y2=bratu(N,N_int,x,Y0=16.*(x[1:-1]-x[1:-1]**2))

In [137]:
fig,ax=plt.subplots(figsize=(7,7))
ax.plot(x,Y1,'^b',ms=7,label='$y_0=(x-x^2)$')
ax.plot(x,Y2,'^r',ms=7,label='$y_0=16(x-x^2)$')
ax.plot(x,Y_anal1,'b',label='$c=0.379$')
ax.plot(x,Y_anal2,'r',label='$c=2.73$')
ax.legend(loc='best',fontsize='small')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 14: équation de la chaleur

${\large \frac{\partial u}{\partial t}= \frac{\partial^2 u}{\partial x^2}}$ pour $0<x<1$

CF: $u(0,t)=u(0,t)=0$

CI: $g(x)= \begin{cases}
 1  \quad a \leq x \leq b \\
 0  \quad \mbox{sinon} \end{cases}$

In [53]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

def Create_Init(x,x1,x2,yy):
    y=np.zeros(np.size(x))
    for i in range(np.size(x)):
        if x[i]>x1 and x[i]<x2:
            y[i]=yy
    return(y)

def tridiag(a,b,c,f):
    N=f.size
    x=np.zeros(N)
    cstar=np.zeros(N)
    astar=a[0]
    x[0]=f[0]/astar
    for k in range(1,N):
        cstar[k-1]=c[k-1]/astar
        astar=a[k]-b[k]*cstar[k-1]
        x[k]=(f[k]-b[k]*x[k-1])/astar
    for k in range(N-2,-1,-1):
        x[k]-=cstar[k]*x[k+1]
    return(x)


def EulAvant(U0,x,tmax,lmb,bc):
    h=x[1]-x[0]
    k=lmb*h**2
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=tmax/(M-1)
    lmb=k/h**(2)
    U=np.zeros((N+2,np.shape(t)[0]))
    U[:,0]=U0
    A=np.diagflat(lmb*np.ones(N-1),-1)+np.diagflat(-(2*lmb+B*k)*np.ones(N))+np.diagflat(+lmb*np.ones(N-1),1)
    for i in range(np.shape(t)[0]-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=Uc+A@Uc
        U[0,i+1]=bc[0]
        U[-1,i+1]=bc[1]
    return(U,lmb,t)

def EulArriere(U0,x,tmax,lmb,bc):
    N=np.shape(x)[0]-2
    h=x[1]-x[0]
    k=lmb*h**2
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=t[1]-t[0]
    lmb=k/h**(2)
    U=np.zeros((N+2,np.shape(t)[0]))
    U[:,0]=U0
    a=(1+2*lmb)*np.ones(N)
    b=-lmb*np.ones(N)
    c=-lmb*np.ones(N)
    for i in range(np.shape(t)[0]-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=tridiag(a,b,c,Uc)
        U[0,i+1]=bc[0]
        U[-1,i+1]=bc[1]
    return(U,lmb,t)

def CrNich(U0,x,tmax,lmb,bc):
    N=np.shape(x)[0]-2
    h=x[1]-x[0]
    k=lmb*h**2
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=t[1]-t[0]
    lmb=k/h**(2)
    U=np.zeros((N+2,np.shape(t)[0]))
    U[:,0]=U0
    B_u=np.diagflat(-lmb*np.ones(N-1),-1)+np.diagflat(-2*(1-lmb)*np.ones(N))+np.diagflat(-lmb*np.ones(N-1),1)
    a=-2*(1+lmb)*np.ones(N)
    b=lmb*np.ones(N)
    c=lmb*np.ones(N)
    for i in range(np.shape(t)[0]-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=tridiag(a,b,c,B_u.dot(Uc))
        U[0,i+1]=bc[0]
        U[-1,i+1]=bc[1]
    return(U,lmb,t)

In [54]:
#Paramètres d'intégration numérique
N=12
x=np.linspace(0.,1.,N+2)
tmax=0.1
#Conditions frontières
bc=[0,0]
#Conditions initiales
a=1./4.
b=3./4.
U0_carre=Create_Init(x,a,b,1)
U0_sinus=np.sin(2*np.pi*x)
lmb=2.5

EAv_carre=EulAvant(U0_carre,x,tmax,lmb,bc)
EAr_carre=EulArriere(U0_carre,x,tmax,lmb,bc)
CN_carre=CrNich(U0_carre,x,tmax,lmb,bc)

EAv_sinus=EulAvant(U0_sinus,x,tmax,lmb,bc)
EAr_sinus=EulArriere(U0_sinus,x,tmax,lmb,bc)
CN_sinus=CrNich(U0_sinus,x,tmax,lmb,bc)


In [55]:
#Solution analytique pour condition initiale carré et sinusoidal
t=EAv_carre[2]
xsol=np.linspace(0,1,100)
Sol_carre=np.zeros((100,np.shape(t)[0]))
Sol_sinus=np.zeros((100,np.shape(t)[0]))
for i in range(np.shape(t)[0]):
    for p in range(1,101):
        lambdap=float(p)*np.pi
        Ap=(2./(lambdap))*(np.cos(lambdap*a)-np.cos(lambdap*b))
        Sol_carre[:,i]+=Ap*np.exp(-lambdap**2*t[i])*np.sin(lambdap*xsol)
        Sol_sinus[:,i]=np.sin(2*np.pi*xsol)*np.exp(-(2*np.pi)**2*t[i])



In [52]:
M=t.shape[0]
Mspan=[0,0.25,0.5,0.75,0.99]
fig,ax=plt.subplots(2,5,figsize=(15,10))
for i in range(len(Mspan)):
    n=int(np.floor(M*Mspan[i]))
    ax[0,i].plot(xsol,Sol_carre[:,n],label='exact')
    ax[0,i].plot(x,EAv_carre[0][:,n],label='Euler avant',marker='^')
    ax[0,i].plot(x,EAr_carre[0][:,n],label='Euler arrière',marker='^')
    ax[0,i].plot(x,CN_carre[0][:,n],label='Crank-Nicholson',marker='^')
    ax[0,i].legend(loc='best',fontsize='small')
    ax[0,i].set_xlabel('x')
    #ax[i].set_ylabel('u')
    ax[0,i].set_ylim((0.,1.2))
    ax[0,i].set_title('time=%.2f, lambda=%.2f'%(EAv_carre[2][n],EAv_carre[1]))
    ax[1,i].plot(xsol,Sol_sinus[:,n],label='exact')
    ax[1,i].plot(x,EAv_sinus[0][:,n],label='Euler avant',marker='^')
    ax[1,i].plot(x,EAr_sinus[0][:,n],label='Euler arrière',marker='^')
    ax[1,i].plot(x,CN_sinus[0][:,n],label='Crank-Nicholson',marker='^')
    ax[1,i].legend(loc='best',fontsize='small')
    ax[1,i].set_xlabel('x')
    #ax[i].set_ylabel('u')
    ax[1,i].set_ylim((-1.2,1.2))
    ax[1,i].set_title('time=%.2f, lambda=%.2f'%(EAv_sinus[2][n],EAv_sinus[1]))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Diffusion radioactive

${\large\frac{\partial u}{\partial t}=D \frac{\partial^{2}u}{\partial x^{2}}-bu}$, où $D$, $b$ sont des constantes positives, $u(0,t)=u(1,t)=0$ et $u(x,0)=g(x)$.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

def Create_Init(x,x1,x2,yy):
    y=np.zeros(np.size(x))
    for i in range(np.size(x)):
        if x[i]>x1 and x[i]<x2:
            y[i]=yy
    return(y)

def tridiag(a,b,c,f):
    N=f.size
    x=np.zeros(N)
    cstar=np.zeros(N)
    astar=a[0]
    x[0]=f[0]/astar
    for k in range(1,N):
        cstar[k-1]=c[k-1]/astar
        astar=a[k]-b[k]*cstar[k-1]
        x[k]=(f[k]-b[k]*x[k-1])/astar
    for k in range(N-2,-1,-1):
        x[k]-=cstar[k]*x[k+1]
    return(x)



def EulArriere(U0,x,h,N,tmax,k,M,B,D,bc):
    t=np.linspace(0,tmax,M)
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    lmb=D*k/h**2
    a=(1+2*lmb+B*k)*np.ones(N)
    b=-lmb*np.ones(N)
    c=-lmb*np.ones(N)
    for i in range(M-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=tridiag(a,b,c,Uc)
        U[0,i+1]=bc[0]
    U[-1,i+1]=bc[1]
    return(U,t,lmb)

def CrNich(U0,x,h,N,T,k,M,B,D,bc):
    t=np.linspace(0,T,M)
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    lmb=D*k/h**2
    B_u=np.diagflat(-lmb*np.ones(N-1),-1)+np.diagflat(-(2-2*lmb-B*k)*np.ones(N))+np.diagflat(-lmb*np.ones(N-1),1)
    a=-(2+2*lmb+B*k)*np.ones(N)
    b=lmb*np.ones(N)
    c=lmb*np.ones(N)
    for i in range(M-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=tridiag(a,b,c,B_u.dot(Uc))
        U[0,i+1]=bc[0]
    U[-1,i+1]=bc[1]
    return(U,t,lmb)

def EulAvant(U0,x,h,N,T,k,M,B,D,bc):
    t=np.linspace(0,T,M)
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    lmb=D*k/h**2
    A=np.diagflat(lmb*np.ones(N-1),-1)+np.diagflat(-(2*lmb+B*k)*np.ones(N))+np.diagflat(+lmb*np.ones(N-1),1)
    for i in range(M-1):
        Uc = U[1:-1,i]
        U[1:-1,i+1]=Uc+A@Uc
        #(-B*Uc+D*laplU)
        U[0,i+1]=bc[0]
    U[-1,i+1]=bc[1]
    return(U,t,lmb)



In [8]:
#Paramètres du problème
L=1.
tmax=0.5
B=1
D=1

#Discrétisation temporelle
M=101
k=tmax/(M-1)

#Discrétisation spatiale
L=1
N=9
h=L/(N+1)
x=np.linspace(0,1,N+2)

#Condition initiale et frontières
U0=np.sin(np.pi*x)
bc=[0,0]




EAv=EulAvant(U0,x,h,N,tmax,k,M,B,D,bc)
EAr=EulArriere(U0,x,h,N,tmax,k,M,B,D,bc)
CN=CrNich(U0,x,h,N,tmax,k,M,B,D,bc)

t=EAv[1]
#Solution analytique
xsol=np.linspace(0,1,10000)
Sol=np.zeros((10000,np.shape(t)[0]))
for i in range(np.shape(t)[0]):	
    Sol[:,i]=np.sin(np.pi*xsol)*np.exp((-(B+(np.pi)**2*D)*t[i]))



In [9]:
M=t.shape[0]
Mspan=[0,0.25,0.5,0.75,0.99]
fig,ax=plt.subplots(1,5,figsize=(15,5))
for i in range(len(Mspan)):
    n=int(np.floor(M*Mspan[i]))
    ax[i].plot(xsol,Sol[:,n],label='exact')
    ax[i].plot(x,EAv[0][:,n],label='Euler avant',marker='^')
    ax[i].plot(x,EAr[0][:,n],label='Euler arrière',marker='^')
    ax[i].plot(x,CN[0][:,n],label='Crank-Nicholson',marker='^')
    ax[i].legend(loc='best',fontsize='small')
    ax[i].set_xlabel('x')
    #ax[i].set_ylabel('u')
    ax[i].set_ylim((-0.1,1.2))
    ax[i].set_title('time=%.2f, lambda= %.2f'%(EAv[1][n],EAv[2]))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
M=[37,297]
N=[9,13]
M=[101,(5/4)*(101-1)+1,(5/1)*(101-1)+1]
N=[9,9,9]
lamb0=[D*(tmax/(M[0]-1))*(N[0]+1)**2,D*(tmax/(M[1]-1))*(N[1]+1)**2,D*(tmax/(M[2]-1))*(N[2]+1)**2]
print(lamb0)

#for j in range(2):
#    for i in range(4):
#        N[j]=2*N[j]+1
#        M[j]=4*M[j]-3
#        print(N[j])
#        print(M[j])
#       print(D*(tmax/(M[j]-1))*(N[j]+1)**2)

[0.5, 0.4, 0.1]


In [19]:
import copy
Err_EAr=[]
Err_EAv=[]
Err_CN=[]
M_axis=[]
lmb=[]

for j in range(1):
    Err_EAr.append([])
    Err_EAv.append([])
    Err_CN.append([])
    lmb.append([])
    M_axis.append([])
    for i in range(3):
        M_axis[j].append(copy.copy(M[j]))
        h=L/(N[j]+1)
        k=tmax/(M[j]-1)
        x=np.linspace(0,1,N[j]+2)
        #Condition initiale et frontières
        U0=np.sin(np.pi*x)
        bc=[0,0]
        #Solution analytique
        Sol=np.sin(np.pi*x)*np.exp((-(B+(np.pi)**2*D)*tmax))
        #Intégration
        U_EAv=EulAvant(U0,x,h,N[j],tmax,k,np.int(M[j]),B,D,bc)
        U_EAr=EulArriere(U0,x,h,N[j],tmax,k,np.int(M[j]),B,D,bc)
        #U_CN=CrNich(U0,x,h,N[j],tmax,k,np.int(M[j]),B,D,bc)
        Err_EAv[j].append(np.linalg.norm(Sol-U_EAv[0][:,-1]))
        Err_EAr[j].append(np.linalg.norm(Sol-U_EAr[0][:,-1]))
        #Err_CN[j].append(np.linalg.norm(Sol-U_CN[0][:,-1]))
        lmb[j].append(D*(k/h**2))
        N[j]=2*N[j]+1
        M[j]=4*M[j]
        

KeyboardInterrupt: 

In [ ]:
print(M_axis)

In [ ]:
mark=['o','s']


fig,ax=plt.subplots(figsize=(15,5))
for i in range(1):
    ax.loglog(M_axis[i],Err_EAv[i],marker=mark[i],label='Erreur Euler avant, lambda=%.2f'%(lamb0[i]))
    ax.loglog(M_axis[i],Err_EAr[i],marker=mark[i],label='Erreur Euler arrière, lambda=%.2f'%(lamb0[i]))
    #ax.loglog(M_axis[i],Err_CN[i],marker=mark[i],label='Erreur Crank-Nicholson, lambda=%.2f'%(lamb0[i]))
    #ax.loglog(M_axis[i],float(M_axis[i])**(-1)*(diff_EAv[0]/float(M_axis[i][0])**(-1)),'b--',label='$1/k$')
ax.legend(loc='best',fontsize='small')
ax.set_xlabel("M")



# Equation d'advection

${\large \frac{\partial u}{\partial t}+a\frac{\partial u}{\partial x}=0} \, \mbox{  pour  } \begin{cases} 0 <x<\infty ,\\ 0 <t \end{cases} $, $u(0,t)=0$

On définit $\lambda =a\frac{k}{h}$

Upwind:

$
u_{i,j+1}=(1-\lambda)u_{i,j}+\lambda u_{i-1,j}
$

Downwind:

$
u_{i,j+1}=(1+\lambda)u_{i,j}-\lambda u_{i+1,j}
$

Lax-Wendroff:

$
u_{i,j+1}=-\frac{1}{2}(1-\lambda)u_{i+1,j}+(1-\lambda^{2}) u_{i,j}+\frac{1}{2}(1+\lambda)u_{i-1,j}
$

In [120]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt


def Create_Init(x,x1,x2,yy):
    y=np.zeros(np.size(x))
    for i in range(np.size(x)):
        if x[i]>x1 and x[i]<x2:
            y[i]=yy
    return(y)

def g(y,a,b):
    k=y.size
    l=b-a
    h=np.zeros(k)
    for p in range(k):
        if y[p]>=(b+a)/2-(b-a)/4 and y[p]<=(b+a)/2+(b-a)/4:
            h[p]=0.5*(1.-np.cos(2.*np.pi*((y[p]-(b+a)/2)/l)))
            h[p]=(1+np.cos(2.*np.pi*((y[p]-(b+a)/2)/(l/2))))*0.5
    return(h)


def integration_adv(U0,a,x,T,lmb,int_mode):
    h=np.abs(x[1]-x[0])
    k=lmb*h/(abs(a))
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=t[1]-t[0]
    lmb=np.abs(a)*k/h
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    #Upwind
    if int_mode=='Upwind':
        for i in range(M-1):
            U[1:-1,i+1]=(1-lmb)*U[1:-1,i]+lmb*U[0:-2,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    elif int_mode=='Downwind':
        for i in range(M-1):
            U[1:-1,i+1]=(1+lmb)*U[1:-1,i]-lmb*U[2:,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    elif int_mode=='Lax-Wendroff':
        A=-(1/2)*lmb*(1-lmb)
        B=(1-lmb**2)
        C=0.5*lmb*(1+lmb)
        for i in range(M-1):
            U[1:-1,i+1]=A*U[2:,i]+B*U[1:-1,i]+C*U[0:-2,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    else:
        print('Unknown integration scheme')	
        U=0
    return(U,x,t)


In [125]:
a=1

lmb=1.
L=120
N=501
x=np.linspace(0.,L,N+2)
tmax=100

U0_carre=Create_Init(x,4,5,1)
a_cos=0
b_cos=10
U0_cos=g(x,a_cos,b_cos)

U_up_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Upwind')
U_down_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Downwind')
U_lw_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Lax-Wendroff')

U_up_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Upwind')
U_down_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Downwind')
U_lw_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Lax-Wendroff')


In [145]:
t=U_up_carre[2]
M=t.shape[0]
Mspan=[0,0.25,0.5,0.75,0.99]

print(U_up_carre[0].shape)
print(M)
fig,ax=plt.subplots(2,5,figsize=(15,7),sharey='row')
for i in range(len(Mspan)):
    n=int(np.floor(M*Mspan[i]))
    center_carre=4.5+a*t[n]
    center_cos=(b_cos+a_cos)/2+a*t[n]
    ax[0,i].plot(x,U_up_carre[0][:,n],label='Upwind',linewidth=4.)
    ax[0,i].plot(x,U_down_carre[0][:,n],label='Downwind',linewidth=3.)
    ax[0,i].plot(x,U_lw_carre[0][:,n],label='Lax-Wendroff',linewidth=2.)
    ax[0,i].plot(x,Create_Init(x-a*t[n],4,5,1),label='Analytique')
    #ax[0,i].set_xlabel('x')
    ax[0,i].set_ylabel('y')
    ax[0,i].set_ylim((-0.5,1.5))
    ax[0,i].set_xlim((center_carre-4,center_carre+4))
    ax[0,i].set_title('Time=%.2f, lambda=%.1f'%(t[n],lmb))
    ax[0,i].legend(loc='best',fontsize='small')
    ax[1,i].plot(x,U_up_cos[0][:,n],label='Upwind',linewidth=4.)
    ax[1,i].plot(x,U_down_cos[0][:,n],label='Downwind',linewidth=3.)
    ax[1,i].plot(x,U_lw_cos[0][:,n],label='Lax-Wendroff',linewidth=2.)
    ax[1,i].plot(x,g(x-a*t[n],0,10),label='Analytique')
    ax[1,i].set_xlabel('x')
    ax[1,i].set_ylabel('y')
    ax[1,i].set_ylim((-0.5,1.5))
    ax[1,i].set_xlim((center_cos-4,center_cos+4))
    ax[1,i].set_title('Time=%.2f, lambda=%.1f'%(t[n],lmb))
    ax[1,i].legend(loc='best',fontsize='small')
    

(601, 1001)
1001


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0
250
500
750
990


Condition CFL
--------
Le domaine de dépendence numérique du schéma doit contenir le domaine de dépendance du problème initial. 
Pour les trois schémas, la condition est la même $\lambda \leq 1$

Stabilité
----
Pour comprende le comportement des différents schémas numériques, il faut effectuer une analyse de stabilité. On trouve alors

${\large u_{i,j+1}=\kappa \, u_{i,j}=|\kappa | e^{I\theta_{n}}}$, où ${\large \theta_{n}=\frac{\Im(\kappa)}{\Re(\kappa)}}$. On appelle $\kappa$ le facteur d'amplification

Considérons une solution de la forme:

${\large u(x,t)=Ue^{I(rx-\omega t)}}$

En remplacant dans l'équation d'advection

${\large U(-I\omega) e^{I(rx-\omega t)}+a (Irx)e^{I(rx-\omega t)}}=0$

${\large \Rightarrow \omega=r a}$ (relation de dispersion). Ce système est non dispersif.

On veut maintenant connaitre l'évolution "vraie" à chaque pas de temps de ma solution analytique.

${\large u(x,t+k)=Ue^{I(rx-\omega (t+k)}=e^{-I\omega k}Ue^{I(rx-\omega t)}=\kappa_{a} Ue^{I(rx-\omega t)}}$

avec ${\large \kappa_{a}=e^{-I\omega k}=e^{-I ark}=e^{-I \lambda r h}=e^{I\theta_{a}}}$. 

Cette expression est à mettre en parallèle avec la première équation pour la solution numérique.

Pour les schémas upwind et downwind:

${\large \kappa =1-\lambda +cos(rh)-\lambda sin(rh) I}$

Pour le schéma Lax-Wendroff:

${\large \kappa= 1-2 \lambda^2 sin^{2}(\frac{rh}{2})-\lambda sin(rh) I}$ 

In [14]:
ko=np.linspace(0.01,np.pi-0.001,100)

lmb=[0.25,0.5,0.75,1.,1.0]
fig,ax=plt.subplots(1,2,figsize=(15,5),subplot_kw=dict(polar=True))
for i in range(5):
    ampl_R=1-lmb[i]+lmb[i]*np.cos(ko)
    ampl_I=-lmb[i]*np.sin(ko)
    ampl=1-lmb[i]+lmb[i]*np.cos(ko)+(1j)*(-lmb[i]*np.sin(ko))
    #ampl=(1+2*lmb[i]*(lmb[i]-1)*(1-np.cos(ko)))**(1/2)
    ratio=np.arctan(ampl_I/ampl_R)/(-lmb[i]*ko)
    ratio=(np.imag(ampl)/np.real(ampl))/(-lmb[i]*ko)
    #ratio=(np.arctan(-(lmb[i]*np.sin(ko))/(1-lmb[i]+lmb[i]*np.cos(ko)))/(-lmb[i]*ko))
    ax[0].plot(ko,np.sqrt(ampl_R**2+ampl_I**2),label='lambda=%.2f'%(lmb[i]))
    #ax[0].plot(ko,np.abs(ampl),label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio,label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,np.abs(ratio),label='lambda=%.2f'%(lmb[i]))
    
ax[0].set_rmax(2)
    #ax.set_rticks([0.0,0.5, 1, 1.5])  # Less radial ticks
ax[0].set_rgrids([0.0,0.5, 1, 1.5],labels=['0.0','0.5', '1', '1.5'],fontsize=20)
ax[0].set_frame_on(False)
ax[0].set_thetamax(180)
ax[0].set_thetagrids(angles=[0,180],labels=['rh=0','rh=$\pi$'],fontsize=15)
#ax[0].annotate('$\lambda =0.5$',xy=[11*np.pi/16, 0.3], fontsize=10)
#ax[0].annotate('$\lambda =0.75$',xy=[5*np.pi/6, 0.85], fontsize=10)
#ax[0].annotate('$\lambda =1.0$',xy=[5*np.pi/6, 1.35], fontsize=10)
#ax[0].annotate('$\lambda =1.25$',xy=[6*np.pi/8, 1.8], fontsize=10)
#ax[0].annotate('$\lambda =1.25$',xy=[3*np.pi/2, 1.8], fontsize=10)
ax[0].legend()
ax[1].set_rmax(2)
    #ax.set_rticks([0.0,0.5, 1, 1.5])  # Less radial ticks
ax[1].set_rgrids([0.0,0.5, 1, 1.5],labels=['0.0','0.5', '1', '1.5'],fontsize=20)
ax[1].set_frame_on(False)
ax[1].set_thetamax(180)
ax[1].set_thetagrids(angles=[0,180],labels=['rh=0','rh=$\pi$'],fontsize=15)
#ax[1].annotate('$\lambda =0.5$',xy=[11*np.pi/16, 0.3], fontsize=10)
#ax[1].annotate('$\lambda =0.75$',xy=[5*np.pi/6, 0.85], fontsize=10)
#ax[1].annotate('$\lambda =1.0$',xy=[5*np.pi/6, 1.35], fontsize=10)
#ax[1].annotate('$\lambda =1.25$',xy=[6*np.pi/8, 1.8], fontsize=10)
#ax[1].annotate('$\lambda =1.25$',xy=[3*np.pi/2, 1.8], fontsize=10)
ax[1].legend()
#ax.annotate('|$\kappa $| = $1+2\lambda (\lambda -1) (1-cos(rh))$',xy=[np.pi/2, 1.8],ha='center', va='bottom', fontsize=30)
#ax.annotate('$rh$',xy=[1*np.pi/100, 1.8], fontsize=15)

#Phase error


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
ko=np.linspace(0.01,np.pi-0.001,100)

lmb=[0.25,0.5,0.75,1.,1.25]
fig,ax=plt.subplots(1,5,figsize=(15,5))
for i in range(5):
    ampl_R=1-lmb[i]+lmb[i]*np.cos(ko)
    ampl_I=-lmb[i]*np.sin(ko)
    ampl=(1-lmb[i]+lmb[i]*np.cos(ko))+(1j)*(-lmb[i]*np.sin(ko))
    #ampl=(1+2*lmb[i]*(lmb[i]-1)*(1-np.cos(ko)))**(1/2)
    ratio0=np.arctan(ampl_I/ampl_R)/(-lmb[i]*ko)
    ratio1=(np.arctan(np.imag(ampl)/np.real(ampl))/(-lmb[i]*ko))
    #ratio=(np.arctan(-(lmb[i]*np.sin(ko))/(1-lmb[i]+lmb[i]*np.cos(ko)))/(-lmb[i]*ko))
    ax[0].plot(ko,np.sqrt(ampl_R**2+ampl_I**2),label='lambda=%.2f'%(lmb[i]))
    #ax[0].plot(ko,np.abs(ampl),label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio,label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio0,label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,np.real(ampl),'o',label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,ampl_R,label='lambda=%.2f'%(lmb[i]))
    ax[2].plot(ko,np.imag(ampl),'o',label='lambda=%.2f'%(lmb[i]))
    ax[2].plot(ko,ampl_I,label='lambda=%.2f'%(lmb[i]))
    ax[3].plot(ko,(np.arctan(np.abs(1/np.real(ampl)))),label='lambda=%.2f'%(lmb[i]))
    ax[3].plot(ko,np.arctan(np.imag(ampl)),label='lambda=%.2f'%(lmb[i]))
    ax[4].plot(ko,ratio0,label='lambda=%.2f'%(lmb[i]))
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
ko=np.linspace(0.01,np.pi-0.001,100)

lmb=[0.25,0.5,0.75,1.]
fig,ax=plt.subplots(1,2,figsize=(15,5),subplot_kw=dict(polar=True))
for i in range(4):
    ampl_R=1-lmb[i]+lmb[i]*np.cos(ko)
    ampl_I=-lmb[i]*np.sin(ko)
    ampl=(-1j*lmb[i]*np.sin(ko)+np.sqrt(1-(lmb[i]**2*np.sin(ko))**2))
    #ampl=(1+2*lmb[i]*(lmb[i]-1)*(1-np.cos(ko)))**(1/2)
    ratio=np.arctan(ampl_I/ampl_R)/(-lmb[i]*ko)
    ratio=(np.imag(ampl)/np.real(ampl))/(-lmb[i]*ko)
    #ratio=(np.arctan(-(lmb[i]*np.sin(ko))/(1-lmb[i]+lmb[i]*np.cos(ko)))/(-lmb[i]*ko))
    #ax[0].plot(ko,np.sqrt(ampl_R**2+ampl_I**2),label='lambda=%.2f'%(lmb[i]))
    ax[0].plot(ko,np.abs(ampl),label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio,label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,np.abs(ratio),label='lambda=%.2f'%(lmb[i]))
    
ax[0].set_rmax(2)
    #ax.set_rticks([0.0,0.5, 1, 1.5])  # Less radial ticks
ax[0].set_rgrids([0.0,0.5, 1, 1.5],labels=['0.0','0.5', '1', '1.5'],fontsize=20)
ax[0].set_frame_on(False)
ax[0].set_thetamax(180)
ax[0].set_thetagrids(angles=[0,180],labels=['rh=0','rh=$\pi$'],fontsize=15)
#ax[0].annotate('$\lambda =0.5$',xy=[11*np.pi/16, 0.3], fontsize=10)
#ax[0].annotate('$\lambda =0.75$',xy=[5*np.pi/6, 0.85], fontsize=10)
#ax[0].annotate('$\lambda =1.0$',xy=[5*np.pi/6, 1.35], fontsize=10)
#ax[0].annotate('$\lambda =1.25$',xy=[6*np.pi/8, 1.8], fontsize=10)
#ax[0].annotate('$\lambda =1.25$',xy=[3*np.pi/2, 1.8], fontsize=10)
ax[0].legend()
ax[1].set_rmax(2)
    #ax.set_rticks([0.0,0.5, 1, 1.5])  # Less radial ticks
ax[1].set_rgrids([0.0,0.5, 1, 1.5],labels=['0.0','0.5', '1', '1.5'],fontsize=20)
ax[1].set_frame_on(False)
ax[1].set_thetamax(180)
ax[1].set_thetagrids(angles=[0,180],labels=['rh=0','rh=$\pi$'],fontsize=15)
#ax[1].annotate('$\lambda =0.5$',xy=[11*np.pi/16, 0.3], fontsize=10)
#ax[1].annotate('$\lambda =0.75$',xy=[5*np.pi/6, 0.85], fontsize=10)
#ax[1].annotate('$\lambda =1.0$',xy=[5*np.pi/6, 1.35], fontsize=10)
#ax[1].annotate('$\lambda =1.25$',xy=[6*np.pi/8, 1.8], fontsize=10)
#ax[1].annotate('$\lambda =1.25$',xy=[3*np.pi/2, 1.8], fontsize=10)
ax[1].legend()
#ax.annotate('|$\kappa $| = $1+2\lambda (\lambda -1) (1-cos(rh))$',xy=[np.pi/2, 1.8],ha='center', va='bottom', fontsize=30)
#ax.annotate('$rh$',xy=[1*np.pi/100, 1.8], fontsize=15)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
ko=np.linspace(0.01,np.pi-0.001,100)

lmb=[0.25,0.5,0.75,1.]
fig,ax=plt.subplots(1,5,figsize=(15,5))
for i in range(4):
    ampl_R=np.sqrt(1-(lmb[i]**2*np.sin(ko))**2)
    ampl_I=-lmb[i]*np.sin(ko)
    ampl=(-1j*lmb[i]*np.sin(ko)+np.sqrt(1-(lmb[i]**2*np.sin(ko))**2))
    #ampl=(1+2*lmb[i]*(lmb[i]-1)*(1-np.cos(ko)))**(1/2)
    ratio0=np.arctan(ampl_I/ampl_R)/(-lmb[i]*ko)
    ratio1=(np.arctan(np.imag(ampl)/np.real(ampl))/(-lmb[i]*ko))
    #ratio=(np.arctan(-(lmb[i]*np.sin(ko))/(1-lmb[i]+lmb[i]*np.cos(ko)))/(-lmb[i]*ko))
    ax[0].plot(ko,np.sqrt(ampl_R**2+ampl_I**2),label='lambda=%.2f'%(lmb[i]))
    #ax[0].plot(ko,np.abs(ampl),label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio,label='lambda=%.2f'%(lmb[i]))
    #ax[1].plot(ko,ratio0,label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,np.real(ampl),'o',label='lambda=%.2f'%(lmb[i]))
    ax[1].plot(ko,ampl_R,label='lambda=%.2f'%(lmb[i]))
    ax[2].plot(ko,np.imag(ampl),'o',label='lambda=%.2f'%(lmb[i]))
    ax[2].plot(ko,ampl_I,label='lambda=%.2f'%(lmb[i]))
    ax[3].plot(ko,(np.arctan(np.abs(1/np.real(ampl)))),label='lambda=%.2f'%(lmb[i]))
    ax[3].plot(ko,np.arctan(np.imag(ampl)),label='lambda=%.2f'%(lmb[i]))
    ax[4].plot(ko,ratio0,label='lambda=%.2f'%(lmb[i]))
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Leapfrog pour l'advection

Un schéma leapfrog pour résoudre l'équation d'advection est  u_{i,j+1}=u_{i,j-1}-\lambda (u_{i+1,j}-u_{i-1,j}).$

In [138]:

import numpy as np
import matplotlib.pyplot as plt


def Create_Init(x,x1,x2,yy):
    y=np.zeros(np.size(x))
    for i in range(np.size(x)):
        if x[i]>x1 and x[i]<x2:
            y[i]=yy
    return(y)
def g(y,a,b):
    k=y.size
    l=b-a
    h=np.zeros(k)
    for p in range(k):
        if y[p]>=(b+a)/2-(b-a)/4 and y[p]<=(b+a)/2+(b-a)/4:
            h[p]=0.5*(1.-np.cos(2.*np.pi*((y[p]-(b+a)/2)/l)))
            h[p]=(1+np.cos(2.*np.pi*((y[p]-(b+a)/2)/(l/2))))*0.5
    return(h)


def Leapfrog_adv(U0,a,x,tmax,lmb):
    h=np.abs(x[1]-x[0])
    k=lmb*h/(abs(a))
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=t[1]-t[0]
    lmb=np.abs(a)*k/h
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    for i in range(M-1):
        if i==0:
            U[1:-1,i+1]=(1-lmb)*U[1:-1,i]+lmb*U[0:-2,i]
            U[0,i+1]=0
            U[-1,i+1]=0
        else:
            U[1:-1,i+1]=U[1:-1,i-1]-lmb*(U[2:,i]-U[0:-2,i])
            U[0,i+1]=0
            U[-1,i+1]=0
    return(U,x,t)



def integration_adv(U0,a,x,T,lmb,int_mode):
    h=np.abs(x[1]-x[0])
    k=lmb*h/(abs(a))
    M=int(np.round(tmax/k))+1
    t=np.linspace(0.,tmax,M)
    k=t[1]-t[0]
    lmb=np.abs(a)*k/h
    U=np.zeros((np.shape(x)[0],np.shape(t)[0]))
    U[:,0]=U0
    #Upwind
    if int_mode=='Upwind':
        for i in range(M-1):
            U[1:-1,i+1]=(1-lmb)*U[1:-1,i]+lmb*U[0:-2,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    elif int_mode=='Downwind':
        for i in range(M-1):
            U[1:-1,i+1]=(1+lmb)*U[1:-1,i]-lmb*U[2:,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    elif int_mode=='Lax-Wendroff':
        A=-(1/2)*lmb*(1-lmb)
        B=(1-lmb**2)
        C=0.5*lmb*(1+lmb)
        for i in range(M-1):
            U[1:-1,i+1]=A*U[2:,i]+B*U[1:-1,i]+C*U[0:-2,i]
            U[0,i+1]=0
            U[-1,i+1]=0
    else:
        print('Unknown integration scheme')	
        U=0
    return(U,x,t)


In [176]:
a=1

lmb=0.9
L=150
N=601
x=np.linspace(0.,L,N+2)
tmax=100

U0_carre=Create_Init(x,4,5,1)
a_cos=0
b_cos=30
U0_cos=g(x,a_cos,b_cos)

U_up_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Upwind')
U_down_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Downwind')
U_lw_carre=integration_adv(U0_carre,a,x,tmax,lmb,'Lax-Wendroff')
U_lp_carre=Leapfrog_adv(U0_carre,a,x,tmax,lmb)

U_up_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Upwind')
U_down_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Downwind')
U_lw_cos=integration_adv(U0_cos,a,x,tmax,lmb,'Lax-Wendroff')
U_lp_cos=Leapfrog_adv(U0_cos,a,x,tmax,lmb)


In [177]:
t=U_up_carre[2]
M=t.shape[0]
Mspan=[0,0.25,0.5,0.75,0.99]


fig,ax=plt.subplots(2,5,figsize=(15,7),sharey='row')
for i in range(len(Mspan)):
    n=int(np.floor(M*Mspan[i]))
    center_carre=4.5+a*t[n]
    center_cos=(b_cos+a_cos)/2+a*t[n]
    ax[0,i].plot(x,U_up_carre[0][:,n],label='Upwind',linewidth=2.)
    ax[0,i].plot(x,U_down_carre[0][:,n],label='Downwind',linewidth=2.)
    ax[0,i].plot(x,U_lw_carre[0][:,n],label='Lax-Wendroff',linewidth=2.)
    ax[0,i].plot(x,U_lp_carre[0][:,n],label='Lax-Wendroff',linewidth=2.)
    ax[0,i].plot(x,Create_Init(x-a*t[n],4,5,1),label='Analytique')
    #ax[0,i].set_xlabel('x')
    ax[0,i].set_ylabel('y')
    ax[0,i].set_ylim((-0.5,1.5))
    ax[0,i].set_xlim((center_carre-4,center_carre+4))
    ax[0,i].set_title('Time=%.2f, lambda=%.1f'%(t[n],lmb))
    ax[0,i].legend(loc='best',fontsize='small')
    ax[1,i].plot(x,U_up_cos[0][:,n],label='Upwind',linewidth=2.)
    ax[1,i].plot(x,U_down_cos[0][:,n],label='Downwind',linewidth=2.)
    ax[1,i].plot(x,U_lw_cos[0][:,n],label='Lax-Wendroff',linewidth=2.)
    ax[1,i].plot(x,U_lp_cos[0][:,n],label='Leapfrog',linewidth=2.)
    ax[1,i].plot(x,g(x-a*t[n],a_cos,b_cos),label='Analytique')
    ax[1,i].set_xlabel('x')
    ax[1,i].set_ylabel('y')
    ax[1,i].set_ylim((-0.5,1.5))
    ax[1,i].set_xlim((center_cos-(b_cos-a_cos)/4,center_cos+(b_cos-a_cos)/4))
    ax[1,i].set_title('Time=%.2f, lambda=%.1f'%(t[n],lmb))
    ax[1,i].legend(loc='best',fontsize='small')
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …